# **SETUP**

In [1]:
import os, sys
import sumolib, traci
#export LD_LIBRARY_PATH=~/Libs/libnsl
#export SUMO_HOME=~/Envs/sumo-env/lib/python3.10/site-packages/sumo
os.environ["LD_LIBRARY_PATH"] = os.path.expanduser("~/Libs/libnsl")
os.environ["SUMO_HOME"] = os.path.expanduser("~/Envs/sumo-env/lib/python3.10/site-packages/sumo")
sys.path.append("/home/hoai-linh.dao/Works/EVCS/Lib-Plan/")
from Vehicles.generateVtypesPassenger import generatePassengerVtypesXML
from Nets.utils import cleanOSM, runAllEdgeChecks, fixOSM, createDisconnectedEdgesPoly

# **cleanOSM + fixOSM** 

In [17]:
cleanOSM("/home/hoai-linh.dao/Works/EVCS/Lib-Plan/Examples/MiniTest/data/marseille.osm", "/home/hoai-linh.dao/Works/EVCS/Lib-Plan/Examples/MiniTest/data/cleaned-marseille.osm")
cleanOSM("/home/hoai-linh.dao/Works/EVCS/Lib-Plan/Examples/MiniTest/data/marseille.osm", "/home/hoai-linh.dao/Works/EVCS/Lib-Plan/Examples/MiniTest/data/only-fixed-marseille.osm")

In [18]:
! netconvert \
  --osm /home/hoai-linh.dao/Works/EVCS/Lib-Plan/Examples/MiniTest/data/only-fixed-marseille.osm \
  -o /home/hoai-linh.dao/Works/EVCS/Lib-Plan/Examples/MiniTest/data/only-fixed-marseille.net.xml \
  --offset.disable-normalization \
  --output.street-names true \
  --output.original-names true 

Success.


In [13]:
! netconvert \
  --osm /home/hoai-linh.dao/Works/EVCS/Lib-Plan/Examples/MiniTest/data/cleaned-marseille.osm \
  -o /home/hoai-linh.dao/Works/EVCS/Lib-Plan/Examples/MiniTest/data/cleaned-marseille.net.xml \
  --offset.disable-normalization \
  --output.street-names true \
  --output.original-names true 

Success.


# **AllEdgeChecks**

In [2]:
runAllEdgeChecks("/home/hoai-linh.dao/Works/EVCS/Lib-Plan/Examples/MiniTest/data/fixed-marseille.net.xml", verbose=False)

Found 5486 disconnected edges.
Found 37298 edges shorter than 5.0 meters.
Found 0 edges with zero lanes.
Found 29620 edges with speed lower than 5.0 m/s.
Found 32 edges with speed higher than 35.0 m/s.
Total edges: 165304, Valid edges: 128006


{'disconnected_edges': 5486,
 'short_edges': 37298,
 'zero_lane_edges': 0,
 'low_speed_edges': 29620,
 'high_speed_edges': 32,
 'total_edges': 165304,
 'valid_edges': 128006}

In [3]:
createDisconnectedEdgesPoly("/home/hoai-linh.dao/Works/EVCS/Lib-Plan/Examples/MiniTest/data/fixed-marseille.net.xml", "/home/hoai-linh.dao/Works/EVCS/Lib-Plan/Examples/MiniTest/data/disconnected_edges.poly.xml")

Found 5486 disconnected edges.
Additional poly file saved to /home/hoai-linh.dao/Works/EVCS/Lib-Plan/Examples/MiniTest/data/disconnected_edges.poly.xml


# **generateVtypesPassenger**

In [10]:
! python /home/hoai-linh.dao/Envs/sumo-env/lib/python3.10/site-packages/sumo/tools/createVehTypeDistribution.py /home/hoai-linh.dao/Works/EVCS/Lib-Plan/Sample-Configs/vtypes-config.txt \
  --output-file /home/hoai-linh.dao/Works/EVCS/Lib-Plan/Sample-Configs/random_passenger_distribution.add.xml \
  --name carDist 

Output written to /home/hoai-linh.dao/Works/EVCS/Lib-Plan/Sample-Configs/random_passenger_distribution.add.xml


In [2]:
! python /home/hoai-linh.dao/Works/EVCS/Lib-Plan/Vehicles/createVTypeDistributionWrapper.py /home/hoai-linh.dao/Works/EVCS/Lib-Plan/Sample-Configs/vtypes-config.txt \
  --output-file /home/hoai-linh.dao/Works/EVCS/Lib-Plan/Sample-Configs/random_passenger_distribution.add.xml \
  --name carDist \
  --size 100 \
  --decimal-places 2 \
  --resampling 50 \
  --seed 24


✔  wrote /home/hoai-linh.dao/Works/EVCS/Lib-Plan/Sample-Configs/random_passenger_distribution.add.xml


# createVTypeDistributionWrapper

In [15]:
! python /home/hoai-linh.dao/Works/EVCS/Lib-Plan/Vehicles/createVTypeDistributionWrapper.py /home/hoai-linh.dao/Works/EVCS/Lib-Plan/Sample-Configs/vtypes-config.txt \
  --evFile /home/hoai-linh.dao/Works/EVCS/AMP-Metropole/Task-3-Completion/data/ev_defination1.xml \
  --output-file /home/hoai-linh.dao/Works/EVCS/Lib-Plan/Sample-Configs/all_vehicle_types.add.xml \
  --name carDist \
  --size 100 \
  --decimal-places 2 \
  --resampling 50 \
  --seed 42


In [3]:
from xml.dom import minidom

doc1 = minidom.parse("/home/hoai-linh.dao/Works/EVCS/Lib-Plan/Sample-Configs/random_passenger_distribution.add.xml")  # target file with <vTypeDistribution>
doc2 = minidom.parse("/home/hoai-linh.dao/Works/EVCS/AMP-Metropole/Task-3-Completion/data/ev_defination1.xml")  # source file with <vType> to merge

dist_elem = doc1.getElementsByTagName("vTypeDistribution")[0]

ev_vtypes = doc2.getElementsByTagName("vType")

defaults = {
    "carFollowModel": "EIDM",
    "tau": 1.0,
    "speedDev": 0.1,
    "length": 4.0,
    "width": 1.7,
    "height": 1.4
}

for vtype in ev_vtypes:
    for attr, val in defaults.items():
        if not vtype.hasAttribute(attr):
            vtype.setAttribute(attr, str(val))

    imported = doc1.importNode(vtype, deep=True)
    dist_elem.appendChild(imported)

def prettify_no_blank_lines(elem):
    raw = elem.toprettyxml(indent="  ")
    return "\n".join([line for line in raw.split("\n") if line.strip()])

output_str = prettify_no_blank_lines(doc1)

with open("/home/hoai-linh.dao/Works/EVCS/Lib-Plan/Sample-Configs/integrated-dist.add.xml", "w") as f:
    f.write(output_str)